# Presentation: Integration of EEMRIO into Brightway 2.5 using EXIOBASE Hybrid Data

by María-Pilar Martínez-Hernando, Thomas Gibon, Roberto Paredes, Luis de la Torre de Palacios, Marcelo F. Ortega, María-Jesús García-Martínez

Supported by: Cátedra Robeco and Ministerio de Universidades

## Introduction: Concept of IO-matrix

Typical structure of a IO-table


| Economic production | Act 1 | Act 2 | Act n | Final demand | Total production |
|---------------------|-------|-------|-------|--------------|------------------|
| Act 1               | z_11  | z_12  | z_1n  | Y_1          | X_1              |
| Act 2               | z_21  | z_22  | z_2n  | Y_2          | X_2              |
| Act n               | z_n1  | z_n2  | z_nn  | Y_n          | X_n              |
| Value added         | V_1   | V_2   | V_n   |              |                  |
| X total             | X_1   | X_2   | X_n   |              |                  |



### Basic concepts

- **Z**: Intermediate consumption

- **Y**: Final demand

- **X**: Total production  
  $$
  X_i = \sum_{j=1}^{n} z_{ij} + Y_i \qquad i = \mathrm{cte}
  $$

- **A**: Technical coefficients  
  $$
  a_{ij} = \frac{z_{ij}}{X_i} \implies z_{ij} = a_{ij} \cdot X_i
  $$

- **L**: Leontief matrix   $$ (\mathbf{I} - \mathbf{A})^{-1}   $$


- **New expression of X:**  
  $$
  \mathbf{X} = (\mathbf{I} - \mathbf{A})^{-1} \cdot \mathbf{Y}
  $$


### Regionalization of matrix

| Production   | Consumption    | Country 1    |    |    | Country 2    |    |    | Country 3    |    |    |
|--------------|---------------|--------------|----|----|--------------|----|----|--------------|----|----|
|              |               | Act 1        | Act 2 | Act 3 | Act 1   | Act 2 | Act 3 | Act 1   | Act 2 | Act 3 |
| **Country 1**| Act 1         |              |      |      |         |        |       |         |        |       |
|              | Act 2         |              |      |      |         |        |       |         |        |       |
|              | Act 3         |              |      |      |         |        |       |         |        |       |
| **Country 2**| Act 1         |              |      |      |         |        |       |         |        |       |
|              | Act 2         |              |      |      |         |        |       |         |        |       |
|              | Act 3         |              |      |      |         |        |       |         |        |       |
| **Country 3**| Act 1         |              |      |      |         |        |       |         |        |       |
|              | Act 2         |              |      |      |         |        |       |         |        |       |
|              | Act 3         |              |      |      |         |        |       |         |        |       |


### Use of IO tables for LCA: Environmentally-Extended Multi-Regional Input-Output Analysis (EEMRIO)

Creation of S matrix

|                 | Country 1   |         |         | Country 2   |         |         | Country 3   |         |         |
|-----------------|-------------|---------|---------|-------------|---------|---------|-------------|---------|---------|
|                 | Act 1       | Act 2   | Act 3   | Act 1       | Act 2   | Act 3   | Act 1       | Act 2   | Act 3   |
| t CO2 eq/t      |             |         |         |             |         |         |             |         |         |
| t CFCs eq/t     |             |         |         |             |         |         |             |         |         |
| mol H+ eq/t     |             |         |         |             |         |         |             |         |         |
| g N eq/t        |             |         |         |             |         |         |             |         |         |
| g P eq/t        |             |         |         |             |         |         |             |         |         |


Total impact is called F : 
$$
\mathbf{F} = \mathbf{S} \cdot \mathbf{X} = \mathbf{S} \cdot (\mathbf{I} - \mathbf{A})^{-1} \cdot \mathbf{Y}
$$

## Objetives

The main problem of EEMRIO is that the common databases (Exiobase, WIOD, EORA) are very large, and they are difficult to integrate with conventional LCA tools

The objetives of this work is to import EXIOBASE 3 hybrid units to Brightway 2.5, exploring two alternatives
1. Creation of a datapackage

2. Creation of a database

## Code created

Init libraries

In [1]:
import pandas as pd #To real CSV and dataframes
import numpy as np #To work with matrix
from typing import List, Tuple, Dict, Callable, Iterable, Union
import itertools
import json
from pathlib import Path

Init brightway

In [3]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw_processing as bwp
from bw_processing.io_helpers import generic_directory_filesystem
import fsspec
from pathlib import Path
import json
import bw2io as bi
from bw2io.importers.io import IOImporter
from bw2io.strategies.io import tidy_tables

print('bw2data version: ',bd.__version__)
print('bw2io version: ',bi.__version__)
print('bw2calc version: ',bc.__version__)

bd.projects
bd.projects.set_current('exiobase_pres')

if any("biosphere3" in db for db in bd.databases): #to confirm biosphere is present
    print('biosphere3 is already present in the project.')
else:
    bi.remote.get_projects()
    bi.remote.install_project('ecoinvent-3.9.1-biosphere', 'Ecoinvent', overwrite_existing=True)

bw2data version:  (4, 5, 1)
bw2io version:  0.9.11
bw2calc version:  2.0.2
03:10:40+0200 [info     ] Applying automatic update: 4.0 database search directories FTS5
03:10:40+0200 [info     ] Reindexing database biosphere3
biosphere3 is already present in the project.


### Read IO data

In [4]:
exiobase_folder="P:/Universidad/Exiobase/" #Put here your folder where you have download Exiobase with /, no with \

Z (intermediate products)

In [6]:
Z_raw=pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_by_product_technology.csv", #You should have this name of file, if not change it
                    index_col = [0,1,2,3,4],
                    header = [0,1,2,3])
Z_raw.index.names=["region","sector","code1","code2","units"]
Z_raw.head() #To see the first lines
Z_reindexed=Z_raw.droplevel([2,3,4], axis=0).copy() #To remove useless rows
Z_reindexed.index.names=["region","sector"]
Z_reindexed.columns=Z_reindexed.index
exio3318_Z=Z_reindexed

F

In [7]:
F_raw = pd.read_excel(exiobase_folder + "MR_HIOT_2011_v3_3_18_extensions_ecoinvent.xlsx", #This data is emissions of intermediate production
                      sheet_name="Emiss_act", #You should have this name of file, if not change it.
                      index_col=[0,1,2,3],
                      header=[0,1,2,3])
F_raw.columns=Z_reindexed.columns
exio3318_emissions_F=F_raw

In [8]:
exio3318_emissions_F

region                                                                                                                              AU  \
sector                                                                                                                      Paddy rice   
Carbon dioxide, fossil                             tonnes       air   Carbon dioxide, fossil                              3.761413e+04   
N2O                                                tonnes       air   Dinitrogen monoxide                                 4.409111e+01   
CH4                                                tonnes       air   Methane, fossil                                     2.652729e+04   
NOX                                                tonnes       air   Nitrogen oxides                                     2.217441e+02   
SOx                                                tonnes       air   Sulfur oxides                                       4.666654e+01   
NH3                                                tonnes       air   Ammonia                                             5.242698e+02   
NMVOC                                              tonnes       air   NMVOC, non-methane volatile organic compounds       5.373167e+01   
CO                                                 tonnes       air   Carbon monoxide, fossil                             1.337744e+02   
Pb                                                 tonnes       air   Lead II                                             4.751886e-03   
Cd                                                 tonnes       air   Cadmium II                                          3.591020e-04   
Hg                                                 tonnes       air   Mercury II                                          2.369422e-04   
As                                                 tonnes       air   Arsenic ion                                         3.929728e-04   
Cr                                                 tonnes       air   Chromium III                                        2.747395e-03   
Cu                                                 tonnes       air   Copper ion                                          2.729743e-02   
Ni                                                 tonnes       air   Nickel II                                           2.502747e-02   
Se                                                 tonnes       air   Selenium IV                                         6.674613e-04   
Zn                                                 tonnes       air   Zinc II                                             2.522927e-02   
PCB                                                tonnes       air   Polychlorinated biphenyls                           3.007777e-06   
PM10                                               tonnes       air   Particulate Matter, > 2.5 um and < 10um             8.783205e+00   
PAH (total of 4 components, sum of EM_AIR.43, 4... tonnes       air   PAH, polycyclic aromatic hydrocarbons               0.000000e+00   
PM2.5                                              tonnes       air   Particulate Matter, < 2.5 um                        1.199926e+01   
Benzo-[a]-pyrene                                   tonnes       air   Benzo(a)pyrene                                      3.810361e-03   
Benzo-[b]-fluoranthene                             tonnes       air   Benzo(b)fluoranthene                                5.194243e-03   
Benzo-[k]-fluoranthene                             tonnes       air   Benzo(k)fluoranthene                                2.197754e-03   
Indeno-[1,2,3-cd]-pyrene                           tonnes       air   Indeno(1,2,3-cd)pyrene                              7.839437e-04   
HCB                                                tonnes       air   Benzene, hexachloro-                                1.102642e-08   
PCDD/F (dioxins and furans)                        tonnes I-TEQ air   Dioxins, measured as 2,3,7,8-tetrachlorodibenzo...  6.705266e-09  

X (total production)

In [9]:
X_raw = pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_principal_production.csv", #You should have this name of file, if not change it
                    header = [0,1,2,3,4,5,6,7])
X_reindexed=X_raw.droplevel([2,3,4,5,6,7], axis=1).copy()
X_reindexed.columns=exio3318_Z.columns #To rename the columns
X_reindexed.index=['Total production'] #To rename the index
exio3318_X=X_reindexed

Calculation of A and L

In [10]:
A=exio3318_Z/exio3318_X.squeeze() #To calculate the matrix A
A=A.fillna(0) #To fill the NaN values with 0
n= len(A.index) #To know the number of rows
Ident=np.eye(n) #To create the identity matrix
I_A=Ident-A #To calculate the matrix I-A

In [11]:
I_A

region                                                              AU  \
sector                                                      Paddy rice   
region sector                                                            
AU     Paddy rice                                         1.000000e+00   
       Wheat                                              0.000000e+00   
       Cereal grains nec                                  0.000000e+00   
       Vegetables, fruit, nuts                            0.000000e+00   
       Oil seeds                                          0.000000e+00   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)      -5.707521e-10   
       Recreational, cultural and sporting services (92)  0.000000e+00   
       Other services (93)                               -9.824484e-11   
       Private households with employed persons (95)      0.000000e+00   
       Extra-territorial organizations and bodies         0.000000e+00   

region                                                                  \
sector                                                           Wheat   
region sector                                                            
AU     Paddy rice                                         0.000000e+00   
       Wheat                                              1.000000e+00   
       Cereal grains nec                                  0.000000e+00   
       Vegetables, fruit, nuts                            0.000000e+00   
       Oil seeds                                          0.000000e+00   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)      -4.466893e-11   
       Recreational, cultural and sporting services (92) -6.332150e-12   
       Other services (93)                               -9.252931e-12   
       Private households with employed persons (95)      0.000000e+00   
       Extra-territorial organizations and bodies         0.000000e+00   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AU     Paddy rice                                             0.000000e+00   
       Wheat                                                  0.000000e+00   
       Cereal grains nec                                      9.929697e-01   
       Vegetables, fruit, nuts                                0.000000e+00   
       Oil seeds                                              0.000000e+00   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)          -1.008281e-10   
       Recreational, cultural and sporting services (92)     -2.340267e-11   
       Other services (93)                                   -5.777404e-11   
       Private households with employed persons (95)          0.000000e+00   
       Extra-territorial organizations and bodies             0.000000e+00   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AU     Paddy rice                                                   0.000000e+00   
       Wheat                                                        0.000000e+00   
       Cereal grains nec                                            0.000000e+00   
       Vegetables, fruit, nuts                                      9.943914e-01   
       Oil seeds                                                    0.000000e+00   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                -9.496865e

Calculation of S

In [12]:
exio3318_emissions_S=exio3318_emissions_F/exio3318_X.squeeze()
exio3318_emissions_S=exio3318_emissions_S.fillna(0)
exio3318_emissions_S=exio3318_emissions_S.replace([np.inf, -np.inf], 0)
exio3318_emissions_S=exio3318_emissions_S*1000 #To have all in kilograms/cubic meters

Index of matrix

In [13]:
n=exio3318_Z.columns.shape[0]
nick=list(range(0,n))
CO2=20001                       #It is necessary to put a numeric value of CO2 emissions. Activities goes to 0 to 7871, so here the number is higher
N2O=20002
CH4=20003
emissions=[CO2, N2O, CH4]

### Creation of datapackage

![Datapackage](img/Datapackage.png)


Technosphere matrix: I-A matrix

Numeric values

In [38]:
tec_data_matrix=I_A.values
tec_data=tec_data_matrix.flatten()
tec_data.shape

print(tec_data.shape)

(61968384,)


Index

In [39]:
tec_index=list(itertools.product(nick, nick))
tec_index=np.array(tec_index, dtype=bwp.INDICES_DTYPE)


print(tec_index.shape)

(61968384,)


Biosphere matrix

Numeric values

In [40]:
b_data=np.array([exio3318_emissions_S.iloc[0,:],exio3318_emissions_S.iloc[1,:],exio3318_emissions_S.iloc[2,:]])
b_data=b_data.flatten()
b_data=b_data.squeeze()
b_data=np.nan_to_num(b_data)

print(b_data.shape)

(23616,)


Index

In [41]:
b_index=list(itertools.product(emissions, nick))
b_index=np.array([b_index],dtype=bwp.INDICES_DTYPE).T
b_index=b_index.squeeze()


print(b_index.shape)
b_index

(23616,)


array([(20001,    0), (20001,    1), (20001,    2), ..., (20003, 7869),
       (20003, 7870), (20003, 7871)],
      dtype=[('row', '<i8'), ('col', '<i8')])

Method

In [42]:
c_data=np.array([1, 273, 29.8]) #For EF V3.1
print(c_data)
c_index=np.array([(CO2, CO2),(N2O,N2O),(CH4,CH4)] ,dtype=bwp.INDICES_DTYPE)
print(c_index.shape)

[  1.  273.   29.8]
(3,)


Create the datapackage

In [43]:
fs = generic_directory_filesystem(dirpath=Path(exiobase_folder))

dp_static = bwp.create_datapackage(
    fs=fs,
    name="exiobase_static",
    matrix_serialize_format_type=bwp.MatrixSerializeFormat.NUMPY
)

In [44]:
dp_static.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=tec_index,
    data_array=tec_data,
)
dp_static.add_persistent_vector(
    matrix='biosphere_matrix',
    indices_array=b_index,
    data_array=b_data,
)
dp_static.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=c_index,
    data_array=c_data,
)

In [45]:
dp_static.finalize_serialization()

Do the LCA

In [ ]:
lca = bc.LCA(
    demand={1063: 1},  
    data_objs=[dp_static],
)
lca.lci()
lca.lcia()
lca.score
print(lca.score)

5766.97817983331


### Database

![Database](img/Database.png)

In [14]:
path_to_intermediate=(Path.cwd()/'example_data')
path_to_intermediate.mkdir(exist_ok=True)

Technosphere matrix

Numeric values

In [15]:
extended_hiot=I_A

for col in extended_hiot.columns:
    extended_hiot[col] = extended_hiot[col].astype(pd.SparseDtype("float", 0))

Index

In [16]:
countries=exio3318_Z.columns.get_level_values(0)
products=exio3318_Z.columns.get_level_values(1)
nick_names=[f"{coun}_{prod}" for coun, prod in zip(countries, products)]
units=np.array(Z_raw.index.get_level_values("units")) #To see the units

Biosphere matrix

Numeric values

In [17]:
extended_S_all=exio3318_emissions_S

for col in extended_S_all.columns:
    extended_S_all[col] = extended_S_all[col].astype(pd.SparseDtype("float", 0))

Index

In [18]:
sustances=extended_S_all.index.get_level_values(0)
compartments=extended_S_all.index.get_level_values(2)
names_ecoinvent=extended_S_all.index.get_level_values(3)
nick_sustances=[f"{sust}_{comp}" for sust, comp in zip(sustances, compartments)]

extended_S=extended_S_all.droplevel([1,2,3], axis=0).copy() #To remove unused columns
extended_S.index=nick_sustances

In [19]:
extended_S

region                                                        AU  \
sector                                                Paddy rice   
Carbon dioxide, fossil_air                          6.118203e+01   
N2O_air                                             7.171728e-02   
CH4_air                                             4.314850e+01   
NOX _air                                            3.606824e-01   
SOx_air                                             7.590640e-02   
NH3_air                                             8.527616e-01   
NMVOC_air                                           8.739834e-02   
CO _air                                             2.175935e-01   
Pb_air                                              7.729277e-06   
Cd_air                                              5.841047e-07   
Hg_air                                              3.854031e-07   
As_air                                              6.391979e-07   
Cr_air                                              4.468831e-06   
Cu_air                                              4.440119e-05   
Ni_air                                              4.070893e-05   
Se_air                                              1.085673e-06   
Zn_air                                              4.103718e-05   
PCB_air                                             4.892360e-09   
PM10_air                                            1.428650e-02   
PAH (total of 4 components, sum of EM_AIR.43, 4...  0.000000e+00   
PM2.5_air                                           1.951765e-02   
Benzo-[a]-pyrene_air                                6.197819e-06   
Benzo-[b]-fluoranthene_air                          8.448801e-06   
Benzo-[k]-fluoranthene_air                          3.574801e-06   
Indeno-[1,2,3-cd]-pyrene_air                        1.275140e-06   
HCB_air                                             1.793525e-11   
PCDD/F (dioxins and furans)_air                     1.090659e-11   
N_water                                             2.154141e+00   
P_water                                             5.106787e-03   
P_soil                                              1.709893e-01   
Zn_soil                                             0.000000e+00   
Pb_soil                                             0.000000e+00   
Carbon dioxide, biogenic_air                        1.555017e-01   
Water Withdrawal Blue_air                           0.000000e+00   

region                                                            \
sector                                                     Wheat   
Carbon dioxide, fossil_air                          8.334139e+01   
N2O_air                                             4.070968e-01   
CH4_air                                             7.023482e-03   
NOX _air                                            5.684470e-01   
SOx_air                                             1.009150e-01   
NH3_air                                             2.330500e+00   
NMVOC_air                                           1.195642e-01   
CO _air                                             2.805936e-01   
Pb_air                                              1.081504e-05   
Cd_air                                              7.640668e-07   
Hg_air                                              4.397526e-07   
As_air                                              7.814986e-07   
Cr_air                                              6.305725e-06   
Cu_air                                              6.185919e-05   
Ni_air                                              5.877865e-05   
Se_air                                              9.560048e-07   
Zn_air                                              5.830881e-05   
PCB_air                                             7.122054e-09   
PM10_air                                            2.054593e-02   
PAH (total of 4 components, sum of EM_AIR.43, 4...  0.000000e+00   
PM2.5_air                                           2.8

Database

In [20]:
product_metadata={
    name: {
        "unit": unit,
        "name": name,
        "location": region,
    }
    for name, unit, region in zip(products, units, countries)
}
biosphere_metadata={
    sustance:{'name':sustance,
               'unit':'kilogram', # not standard units
               'compartment':compartment,
    }
    for sustance, compartment in zip(nick_sustances,compartments)
}
biosphere_metadata['Water Withdrawal Blue_air']['unit'] = 'cubic meter' #water has other units

metadata_dict = {**product_metadata, **biosphere_metadata}

with open(path_to_intermediate/'io_metadata.json', 'w') as fp:
    json.dump(metadata_dict, fp,indent=4)

In [21]:
biosphere_metadata

{'Carbon dioxide, fossil_air': {'name': 'Carbon dioxide, fossil_air',
  'unit': 'kilogram',
  'compartment': 'air'},
 'N2O_air': {'name': 'N2O_air', 'unit': 'kilogram', 'compartment': 'air'},
 'CH4_air': {'name': 'CH4_air', 'unit': 'kilogram', 'compartment': 'air'},
 'NOX _air': {'name': 'NOX _air', 'unit': 'kilogram', 'compartment': 'air'},
 'SOx_air': {'name': 'SOx_air', 'unit': 'kilogram', 'compartment': 'air'},
 'NH3_air': {'name': 'NH3_air', 'unit': 'kilogram', 'compartment': 'air'},
 'NMVOC_air': {'name': 'NMVOC_air', 'unit': 'kilogram', 'compartment': 'air'},
 'CO _air': {'name': 'CO _air', 'unit': 'kilogram', 'compartment': 'air'},
 'Pb_air': {'name': 'Pb_air', 'unit': 'kilogram', 'compartment': 'air'},
 'Cd_air': {'name': 'Cd_air', 'unit': 'kilogram', 'compartment': 'air'},
 'Hg_air': {'name': 'Hg_air', 'unit': 'kilogram', 'compartment': 'air'},
 'As_air': {'name': 'As_air', 'unit': 'kilogram', 'compartment': 'air'},
 'Cr_air': {'name': 'Cr_air', 'unit': 'kilogram', 'compartme

In [22]:
flows_code=[]
print(len(nick_sustances))
print(len(names_ecoinvent))
print(len(compartments))

34
34
34


In [25]:
value=[act for act in bd.Database('biosphere3') if act['name']==names_ecoinvent[0] and act['categories'] == (compartments[0],)]
print(value)

['Carbon dioxide, fossil' (kilogram, None, ('air',))]


In [26]:
flows_code=[]
for i in range(len(nick_sustances)):
    value=[act for act in bd.Database('biosphere3') if act['name']==names_ecoinvent[i] and act['categories'] == (compartments[i],)]
    codes=value[0]['code']
    flows_code.append(codes)
flows_code_pd=pd.DataFrame(flows_code)
flows_code_pd.index=nick_sustances

In [27]:
biosphere_mapping={}
for name, code in zip(nick_sustances, flows_code):
    biosphere_mapping[name]=code

In [28]:
biosphere_mapping

{'Carbon dioxide, fossil_air': '349b29d1-3e58-4c66-98b9-9d1a076efd2e',
 'N2O_air': '20185046-64bb-4c09-a8e7-e8a9e144ca98',
 'CH4_air': '0795345f-c7ae-410c-ad25-1845784c75f5',
 'NOX _air': 'c1b91234-6f24-417b-8309-46111d09c457',
 'SOx_air': 'ba5fc0b6-770b-4da1-9b3f-e3b5087f07cd',
 'NH3_air': '87883a4e-1e3e-4c9d-90c0-f1bea36f8014',
 'NMVOC_air': 'd3260d0e-8203-4cbb-a45a-6a13131a5108',
 'CO _air': 'ba2f3f82-c93a-47a5-822a-37ec97495275',
 'Pb_air': '8e123669-94d3-41d8-9480-a79211fe7c43',
 'Cd_air': '1c5a7322-9261-4d59-a692-adde6c12de92',
 'Hg_air': '71234253-b3a7-4dfe-b166-a484ad15bee7',
 'As_air': 'dc6dbdaa-9f13-43a8-8af5-6603688c6ad0',
 'Cr_air': 'e142b577-e934-4085-9a07-3983d4d92afb',
 'Cu_air': 'ec8144d6-d123-43b1-9c17-a295422a0498',
 'Ni_air': 'a5506f4b-113f-4713-95c3-c819dde6e48b',
 'Se_air': '454c61fd-c52b-4a04-9731-f141bb7b5264',
 'Zn_air': '5ce378a0-b48d-471c-977d-79681521efde',
 'PCB_air': 'c9b97088-efc6-43c1-8d26-f78d6cdbb50b',
 'PM10_air': 'b967e1bf-f09b-4c89-8740-ace21db47bba'

Create database

In [29]:
name_database="exiobase_318_hybrid" 
if name_database in bd.databases:
    borro=input('Do you want to delete the existing database? (y/n): ').lower()
    if borro=='y':
        del bd.databases[name_database]
        tidy_tables(extended_hiot,extended_S,path_to_intermediate)
        exiobase_bw= IOImporter(dirpath=path_to_intermediate,
                         db_name='exiobase_318_hybrid',
                         b3mapping=biosphere_mapping)
        exiobase_bw.apply_strategies()
        exiobase_bw.write_database()
    else:
        print('Database is installed')
else:
    tidy_tables(extended_hiot,extended_S,path_to_intermediate)
    exiobase_bw= IOImporter(dirpath=path_to_intermediate,
                         db_name='exiobase_318_hybrid',
                         b3mapping=biosphere_mapping)
    exiobase_bw.apply_strategies()
    exiobase_bw.write_database()

100%|██████████| 7872/7872 [00:01<00:00, 5288.08it/s]


03:18:33+0200 [info     ] Vacuuming database            
03:18:36+0200 [info     ] Starting IO table write       
03:18:36+0200 [info     ] Adding technosphere matrix    
03:21:23+0200 [info     ] Adding biosphere matrix       
03:21:26+0200 [info     ] Finalizing serialization      


Use database

In [30]:
test= bd.Database('exiobase_318_hybrid')

In [31]:
locations = {a['location'] for a in test}
names = {a['name'] for a in test}

In [32]:
method_chosen=('EF v3.1', 'climate change', 'global warming potential (GWP100)')

In [33]:
if method_chosen in bd.methods:
    print("the method is in the list")
else:
    print("the method is not in the list")

the method is in the list


In [34]:
act2=next((a for a in test if a['name'] == 'Copper products' and a['location'] == 'CN'), None)

In [35]:
lca = bc.LCA(
    demand={act2: 1},
    method=method_chosen
)
lca.lci()
lca.lcia()
lca.score
print(lca.score)

5766.978043725156


In [36]:
lca.to_dataframe().pivot_table(index='row_name',aggfunc='sum',values='amount')

,amount
row_name,
"Carbon dioxide, fossil",4797.097486
Dinitrogen monoxide,83.360534
"Methane, fossil",639.128233


Quick check

In [47]:
L=np.linalg.inv(I_A)
M=exio3318_emissions_S @ L
c_series=pd.Series(
    c_data,
    index=[
        ("Carbon dioxide, fossil", "tonnes", "air","Carbon dioxide, fossil"),
        ("N2O", "tonnes", "air","Dinitrogen monoxide"),
        ("CH4", "tonnes", "air","Methane, fossil"),
    ],
)
# Footprints
footprints=c_series@M.loc[c_series.index]
footprints[1063]

5766.978179829129

## Conclusions and future work

1. It is possible to create a database with Exiobase and combing with other databases, with time efficiency

2. Trying to adapt the database created to Activity browser, making easier the integration of EEMRIO in conventional tools


# Thanks for your attention

Contact: mdp.martinez@upm.es